# Task 1 - Beam Search To Predict Chess Move

In [3]:
import heapq
import random


class SimpleChess:
    def __init__(self):
        self.turn = 'white'  
        self.board = self.initialize_board()
        self.move_history = []

    def initialize_board(self):
        return [
            ['r', 'n', 'b', 'q', 'k', 'b', 'n', 'r'],
            ['p', 'p', 'p', 'p', 'p', 'p', 'p', 'p'],
            ['.', '.', '.', '.', '.', '.', '.', '.'],
            ['.', '.', '.', '.', '.', '.', '.', '.'],
            ['.', '.', '.', '.', '.', '.', '.', '.'],
            ['.', '.', '.', '.', '.', '.', '.', '.'],
            ['P', 'P', 'P', 'P', 'P', 'P', 'P', 'P'],
            ['R', 'N', 'B', 'Q', 'K', 'B', 'N', 'R']
        ]

    def copy(self):
        new_game = SimpleChess()
        new_game.turn = self.turn
        new_game.board = [row[:] for row in self.board]
        new_game.move_history = self.move_history[:]
        return new_game

    def generate_legal_moves(self):
        moves = []
        for i in range(8):
            for j in range(8):
                piece = self.board[i][j]
                if (self.turn == 'white' and piece.isupper()) or (self.turn == 'black' and piece.islower()):
                    if piece.lower() == 'p':  
                        if self.turn == 'white' and i > 0 and self.board[i-1][j] == '.':
                            moves.append(((i, j), (i-1, j)))
                        elif self.turn == 'black' and i < 7 and self.board[i+1][j] == '.':
                            moves.append(((i, j), (i+1, j)))
                    elif piece.lower() == 'n': 
                        for di, dj in [(-2,-1), (-2,1), (-1,-2), (-1,2), (1,-2), (1,2), (2,-1), (2,1)]:
                            ni, nj = i + di, j + dj
                            if 0 <= ni < 8 and 0 <= nj < 8:
                                target = self.board[ni][nj]
                                if target == '.' or (self.turn == 'white' and target.islower()) or (self.turn == 'black' and target.isupper()):
                                    moves.append(((i, j), (ni, nj)))
        return moves

    def make_move(self, move):
        (i1, j1), (i2, j2) = move
        self.board[i2][j2] = self.board[i1][j1]
        self.board[i1][j1] = '.'
        self.turn = 'black' if self.turn == 'white' else 'white'
        self.move_history.append(move)

    def is_game_over(self):

        white_king = any('K' in row for row in self.board)
        black_king = any('k' in row for row in self.board)
        return not (white_king and black_king)

def evaluate_board(board):
    piece_values = {
        'P': 1, 'p': -1,
        'N': 3, 'n': -3,
        'B': 3, 'b': -3,
        'R': 5, 'r': -5,
        'Q': 9, 'q': -9,
        'K': 0, 'k': 0
    }
    
    score = 0
    for row in board:
        for piece in row:
            if piece in piece_values:
                score += piece_values[piece]
    return score

def beam_search_chess(board, beam_width=3, depth_limit=3):
    beam = [(0, [], board.copy())]
    
    for _ in range(depth_limit):
        candidates = []
        
        for eval_score, move_sequence, current_board in beam:
            if current_board.is_game_over():
                candidates.append((eval_score, move_sequence, current_board))
                continue
                
            for move in current_board.generate_legal_moves():
                new_board = current_board.copy()
                new_board.make_move(move)
                
                new_eval = evaluate_board(new_board.board)
                if new_board.turn == 'black':  
                    new_eval = -new_eval 
                
                new_move_sequence = move_sequence + [move]
                candidates.append((new_eval, new_move_sequence, new_board))
        
        if not candidates:
            break
            
        beam = heapq.nlargest(beam_width, candidates, key=lambda x: x[0])
    
    if not beam:
        return None, 0
        
    best_eval, best_moves, _ = max(beam, key=lambda x: x[0])
    return best_moves, best_eval


game = SimpleChess()
beam_width = 3
depth_limit = 2

print("Initial board:")
for row in game.board:
    print(' '.join(row))
print()

best_moves, eval_score = beam_search_chess(game, beam_width, depth_limit)

if best_moves:
    print(f"Best move sequence found (evaluation: {eval_score}):")
    for i, move in enumerate(best_moves, 1):
        print(f"{i}. {move}")
        
    # Show the board after first move
    if best_moves:
        game.make_move(best_moves[0])
        print("\nBoard after first move:")
        for row in game.board:
            print(' '.join(row))
else:
    print("No legal moves found.")

Initial board:
r n b q k b n r
p p p p p p p p
. . . . . . . .
. . . . . . . .
. . . . . . . .
. . . . . . . .
P P P P P P P P
R N B Q K B N R

Best move sequence found (evaluation: 0):
1. ((6, 0), (5, 0))
2. ((0, 1), (2, 0))

Board after first move:
r n b q k b n r
p p p p p p p p
. . . . . . . .
. . . . . . . .
. . . . . . . .
P . . . . . . .
. P P P P P P P
R N B Q K B N R


# Task 2 - Hill Climbing Algorithm To Find The Shortest Delivery Route

In [13]:
import random
import math

def euclidean_distance(p1, p2):
    return math.sqrt((p1[0] - p2[0]) ** 2 + (p1[1] - p2[1]) ** 2)

def calculate_total_distance(route):
    distance = 0
    for i in range(len(route) - 1):
        distance += euclidean_distance(route[i], route[i + 1])
    return distance

def get_neighbor(route):
    neighbor = route.copy()
    i, j = random.sample(range(len(route)), 2)
    neighbor[i], neighbor[j] = neighbor[j], neighbor[i]
    return neighbor

def hill_climbing(coordinates, max_iterations=1000):
    current_route = coordinates.copy()
    random.shuffle(current_route)
    current_cost = calculate_total_distance(current_route)

    best_route = current_route.copy()
    best_cost = current_cost

    for _ in range(max_iterations):
        neighbor_route = get_neighbor(current_route)
        neighbor_cost = calculate_total_distance(neighbor_route)

        if neighbor_cost < current_cost:
            current_route = neighbor_route
            current_cost = neighbor_cost

            if current_cost < best_cost:
                best_route = current_route.copy()
                best_cost = current_cost

    return best_route, best_cost
    
delivery_points = [(0, 0), (1, 5), (4, 3), (10, 2), (7, 8)]
route, distance = hill_climbing(delivery_points)

print("Optimized Delivery Route:")
for point in route:
    print(point)
print(f"Total Distance: {distance:.2f}")


Optimized Delivery Route:
(10, 2)
(7, 8)
(4, 3)
(1, 5)
(0, 0)
Total Distance: 21.24


# Task 3 - Genetic Algorithm To Solve the TSP (Traveling Salesman Problem)

In [15]:
import random
import math

# Generate 10 random cities
def generate_cities(n=10, x_range=(0, 100), y_range=(0, 100)):
    return [(random.randint(*x_range), random.randint(*y_range)) for _ in range(n)]

def euclidean_distance(a, b):
    return math.sqrt((a[0] - b[0])**2 + (a[1] - b[1])**2)

def total_distance(route):
    dist = 0
    for i in range(len(route)):
        dist += euclidean_distance(route[i], route[(i + 1) % len(route)])
    return dist

def initial_population(cities, size):
    return [random.sample(cities, len(cities)) for _ in range(size)]

def fitness(route):
    return 1 / total_distance(route)

def tournament_selection(pop, k=3):
    selected = random.sample(pop, k)
    return min(selected, key=total_distance)

def ordered_crossover(parent1, parent2):
    size = len(parent1)
    start, end = sorted(random.sample(range(size), 2))
    child = [None] * size
    child[start:end+1] = parent1[start:end+1]
    fill = [city for city in parent2 if city not in child]
    pointer = 0
    for i in range(size):
        if child[i] is None:
            child[i] = fill[pointer]
            pointer += 1
    return child

def mutate(route, mutation_rate=0.1):
    for i in range(len(route)):
        if random.random() < mutation_rate:
            j = random.randint(0, len(route)-1)
            route[i], route[j] = route[j], route[i]

def evolve(population, elite_size=1, mutation_rate=0.1):
    next_gen = sorted(population, key=total_distance)[:elite_size]
    while len(next_gen) < len(population):
        parent1 = tournament_selection(population)
        parent2 = tournament_selection(population)
        child = ordered_crossover(parent1, parent2)
        mutate(child, mutation_rate)
        next_gen.append(child)
    return next_gen

def genetic_algorithm(cities, population_size=100, generations=500):
    pop = initial_population(cities, population_size)
    best_route = min(pop, key=total_distance)
    for _ in range(generations):
        pop = evolve(pop)
        current_best = min(pop, key=total_distance)
        if total_distance(current_best) < total_distance(best_route):
            best_route = current_best
    return best_route, total_distance(best_route)
    
cities = generate_cities(10)
route, distance = genetic_algorithm(cities)

print("Best Route Found:")
for city in route:
    print(city)
print(f"Total Distance: {distance:.2f}")


Best Route Found:
(82, 13)
(34, 14)
(23, 7)
(27, 40)
(29, 46)
(0, 47)
(2, 55)
(32, 67)
(47, 75)
(68, 38)
Total Distance: 258.39
